# Aquisição de dados no portal da ONS Dados Aberto

https://ons-aws-prod-opendata.s3.amazonaws.com/

https://registry.opendata.aws/ons-opendata-portal/

https://github.com/awslabs/open-data-registry/blob/main/datasets/ons-opendata-portal.yaml#L17

# Importanto bibliotecas

In [1]:
import pandas as pd
import plotly.graph_objects as go
import requests
import time
from io import StringIO

In [2]:
# Definição de parâmetros
plot = False
data_limite = '2025-06-30'

# Funções

In [3]:
def preencher_horas_faltantes(df, metodo='spline', order=2):
    """
    Preenche as horas faltantes no índice datetime do DataFrame, interpolando os valores de acordo com o método escolhido.

    Parâmetros:
    df (pd.DataFrame): DataFrame com índice datetime e colunas numéricas.
    metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
                  'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
                  'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
                  'akima', 'cubicspline', 'from_derivatives').
                  Padrão: 'spline'.
    order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Retorna:
    pd.DataFrame: DataFrame com as horas corrigidas e valores interpolados.
    """

    if df.empty:
        print("O DataFrame está vazio. Nenhuma interpolação foi realizada.")
        return df

    # Garante que o índice é datetime e remove valores inválidos
    df = df.copy()
    df.index = pd.to_datetime(df.index, errors='coerce')

    if df.index.isna().any():
        raise ValueError(
            "O índice contém valores não reconhecidos como datas.")

    # Criar um range de tempo contínuo com frequência de 1 hora
    full_range = pd.date_range(
        start=df.index.min(), end=df.index.max(), freq='h')

    # Só reindexa se houver horas ausentes
    if not df.index.equals(full_range):
        df = df.reindex(full_range)

    # Filtra apenas colunas numéricas
    numeric_cols = df.select_dtypes(include=['number']).columns
    non_numeric_cols = df.select_dtypes(exclude=['number']).columns

    if numeric_cols.empty:
        print("Nenhuma coluna numérica encontrada para interpolação. Retornando DataFrame original.")
        return df

    # Lista de métodos válidos
    metodos_validos = [
        'linear', 'time', 'index', 'values', 'pad', 'nearest', 'zero',
        'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial', 'krogh',
        'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline', 'from_derivatives'
    ]

    # Aplicando o método de interpolação escolhido
    try:
        if metodo in metodos_validos:
            if metodo in ['spline', 'polynomial'] and order is None:
                order = 2  # Define um padrão se o usuário não passar
            df[numeric_cols] = df[numeric_cols].interpolate(
                method=metodo, order=order if metodo in ['spline', 'polynomial'] else None)
        else:
            raise ValueError(
                f"Método de interpolação inválido: '{metodo}'. Métodos disponíveis: {', '.join(metodos_validos)}")
    except Exception as e:
        print(f"Erro ao interpolar os dados: {e}")
        return df

    # Aviso se existirem colunas não numéricas
    if not non_numeric_cols.empty:
        print(
            f"As colunas não numéricas foram ignoradas: {list(non_numeric_cols)}")

    return df

In [4]:
def get_carga_ons(  start: pd.Timestamp,
                    end: pd.Timestamp,
                    extension: str = 'csv',
                    max_retries: int = 3,
                    metodo='spline',
                    order=2) -> pd.DataFrame:  # type: ignore
    """
    Baixa arquivos públicos do ONS de um dataset específico em um intervalo de datas, com tentativas automáticas em caso de falha.

    Parâmetros:
    ----------
    start (pd.Timestamp): Data de início no formato Timestamp.
    end (pd.Timestamp): Data de fim no formato Timestamp.
    extension (str): Extensão do arquivo a ser baixado. ".csv" ou ".xlsx" ou ".parquet".
    max_retries (int): Número máximo de tentativas antes de desistir.
    metodo (str): Método de interpolação ('linear', 'time', 'index', 'values', 'pad', 
        'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 
        'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 
        'akima', 'cubicspline', 'from_derivatives').
        Padrão: 'spline'.
    order (int, opcional): Ordem do polinômio para os métodos 'polynomial' e 'spline'. Padrão: 2.

    Retorno:
    -------
    pd.DataFrame: DataFrame contendo os valores de carga com timestamps em UTC.
    """
    
    # Garante que os argumentos start e end são timestamps corretamente formatados
    start = pd.Timestamp(start).tz_localize('UTC') if pd.Timestamp(start).tz is None else pd.Timestamp(start)
    end = pd.Timestamp(end).tz_localize('UTC') if pd.Timestamp(end).tz is None else pd.Timestamp(end)
       
    # Inicializa 'Cargas' como um dataFrame vazio
    cargas = pd.DataFrame()
    
    for year in range(start.year, end.year + 1):
        attempts = 0
        while attempts < max_retries:
            try:
                # Obtenção dos dados
                url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/curva-carga-ho/CURVA_CARGA_{year}.{extension}"
                response = requests.get(url)
                data = pd.read_csv(StringIO(response.text), sep=';')
                
                # Pivotar os dados para o formato largo
                data = data.pivot(index='din_instante', columns='nom_subsistema', values='val_cargaenergiahomwmed')
                
                # Converter o índice para datetime             
                data.index = pd.to_datetime(data.index)
                
                # Preencher possíveis valores ausentes antes da interpolação
                data.ffill(inplace=True)

                # Preenchimento de horas faltantes usando spline de ordem 2
                data = preencher_horas_faltantes(data, metodo=metodo, order=order)

                # Resetar o índice e nomear corretamente a coluna de timestamp
                data = data.reset_index(names=['Timestamp'])
                                
                # Aviso de que os dados foram baixados com sucesso
                print(f"Dados de {year} adquiridos com sucesso.")
                
                # Adiciona o DataFrame ao DataFrame de cargas
                cargas = pd.concat([cargas, data], ignore_index=True).drop_duplicates(subset='Timestamp', keep='last')

                break
            except Exception as e:
                attempts += 1
                print(
                    f"Erro ao obter os dados de {start.year} (Tentativa {attempts}/{max_retries}): {e}")

                if attempts < max_retries:
                    print("Tentando novamente em 5 segundos...")
                    time.sleep(5)
                else:
                    print(
                        f"Falha após {max_retries} tentativas. Pulando ano {start.year}.")     
    
    cargas['Timestamp'] = pd.to_datetime(cargas['Timestamp'])
    cargas['Timestamp'] = cargas['Timestamp'].dt.tz_localize('Etc/GMT+3')           
    return cargas

In [5]:
def plot_dataset_plotly(df, title="Gráfico Interativo com Plotly"):
    fig = go.Figure()
    for col in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col))
    fig.update_layout(
        title=title,
        xaxis_title="Tempo",
        yaxis_title="Valores",
        hovermode='x unified',
        template='plotly_white'
    )
    fig.show()

In [6]:
def filter_by_date_and_save(
    file_path,
    limit_date=None,
    timezone='Europe/Paris'
):
    """
    Lê o CSV de preços day-ahead com coluna 'Timestamp' contendo timezone,
    converte para o timezone desejado, filtra até a data limite (exclusivo)
    e sobrescreve o arquivo original.

    Parâmetros:
    - file_path (str): Caminho para o arquivo CSV.
    - limit_date (str): Data limite no formato 'YYYY-MM-DD' (exclusivo).
    - timezone (str): Timezone desejado (ex: 'Europe/Paris').
    """
    # Se a data limite não for fornecida a função não faz nada
    if limit_date is None:
        print("Nenhuma data limite fornecida. Nenhum filtro aplicado.")
        return

    # Lê o CSV e converte 'Timestamp' para datetime com UTC
    df = pd.read_csv(file_path, parse_dates=['Timestamp'])
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True).dt.tz_convert(timezone)

    # Aplica filtro até a data limite no timezone especificado
    limit_date_tz = pd.Timestamp(limit_date, tz=timezone)
    df = df[df['Timestamp'] < limit_date_tz]

    # Sobrescreve o arquivo original
    df.to_csv(file_path, index=False)

## Carga ONS

In [7]:
# Verifica se o csv já existe
try:
    df = pd.read_csv('./../data/carga_ons.csv')
    print("Arquivo CSV encontrado.")
    if plot:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)
        df.set_index('Timestamp', inplace=True)
        plot_dataset_plotly(df)
except FileNotFoundError:
    # Aquisição das cargas da ONS de 2000 até o disponível em 2025
    carga_ons = get_carga_ons(
        start = pd.Timestamp('2000-01-01'),
        end = pd.Timestamp('2025-12-31'),
        extension='csv',
        order=3
    )
    # Concatenar todos os DataFrames, tratando duplicatas pela coluna Timestamp
    if carga_ons is not None and not carga_ons.empty:
        carga_ons.to_csv('./../data/carga_ons.csv', index=False)
        print("Arquivo CSV salvo com sucesso.")
        filter_by_date_and_save(
            file_path='./../data/carga_ons.csv',
            limit_date=data_limite,
            timezone='America/Sao_Paulo')
        if plot:
            carga_ons['Timestamp'] = pd.to_datetime(carga_ons['Timestamp'])
            carga_ons.set_index('Timestamp', inplace=True)
            plot_dataset_plotly(carga_ons)
    else:
        print("Nenhum dado foi adquirido, arquivo CSV não gerado.")

Dados de 2000 adquiridos com sucesso.
Dados de 2001 adquiridos com sucesso.
Dados de 2002 adquiridos com sucesso.
Dados de 2003 adquiridos com sucesso.
Dados de 2004 adquiridos com sucesso.
Dados de 2005 adquiridos com sucesso.
Dados de 2006 adquiridos com sucesso.
Dados de 2007 adquiridos com sucesso.
Dados de 2008 adquiridos com sucesso.
Dados de 2009 adquiridos com sucesso.
Dados de 2010 adquiridos com sucesso.
Dados de 2011 adquiridos com sucesso.
Dados de 2012 adquiridos com sucesso.
Dados de 2013 adquiridos com sucesso.
Dados de 2014 adquiridos com sucesso.
Dados de 2015 adquiridos com sucesso.
Dados de 2016 adquiridos com sucesso.
Dados de 2017 adquiridos com sucesso.
Dados de 2018 adquiridos com sucesso.
Dados de 2019 adquiridos com sucesso.
Dados de 2020 adquiridos com sucesso.
Dados de 2021 adquiridos com sucesso.
Dados de 2022 adquiridos com sucesso.
Dados de 2023 adquiridos com sucesso.
Dados de 2024 adquiridos com sucesso.
Dados de 2025 adquiridos com sucesso.
Arquivo CSV 